<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#CA6F1E;
            overflow:hidden;
            font-weight:700"><span style='color:#283747'></span>"Predict behavior to retain customers." </div>

> * Information **DATA**
> https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

- [1. Exploring The Dataset](#a)
    - [1.1 Load The Datset](#1.1)
    - [1.2 Data Description](#1.2)
    - [1.3 Data Cleaning](#1.3)
        - [1.3.1 Missing Value](#1.3.1)
        - [1.3.2 Duplicated](#1.3.2)
        - [1.3.3 Outliter](#1.3.3)
- [2.Data Visualization and Exploratory Data Anlysis(EDA)](#2)
    - [2.1 Gender, Seniority, Partner and Pependends](#2.1)
    - [2.2 Services](#2.2)
    - [2.3 Security_support](#2.3)
    - [2.4 Contract type](#2.4)
    - [2.5 Churn analysis based on paying options](#2.5)
    - [2.6 Churn analysis based on tenure group](#2.6)
    - [2.7 Charges](#2.7)
- [3. Data preprocessing and feature engineering](#4)
    - [3.1 Min-Max Scaling or Standardization?](#4.1)
    - [3.2 Encoding Categorical Variables](#4.2)
        - [3.2.1 Label Encoding](#4.2.1)
        - [3.2.2 One hot Encoding](#4.2.2)
        - [3.2.3 Multicolinearity](#4.2.3)
- [4. Data Splitting and Model Building](#5)
   - [4.1 Data Splitting](#5.1)
   - [4.2 Model Buiding](#5.2)
       - [4.2.1 Feature selection](#5.2.1)
           - [4.2.1.1 Recursive Feature Elimination (RFE)](#5.2.1.1)
           - [4.2.1.2 Model building using L1 Regularization](#5.2.1.2)
       - [4.2.2 Feature importance](#5.2.2)
- [5. Model Evaluation](#6) 


<a id="a"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#222222;
            overflow:hidden;
            font-weight:700">1 <span style='color:#CDA63A'>|</span>Exploring The Dataset</div>

In [10]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler , OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

### check multicorre
## Feature selection
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")
# Display all the column of the dataframes
pd.set_option('display.max_columns', None)

<a id="1.1"></a>
## <b>1.1 <span style='color:#00008B'>Load The dataset</span></b>

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/ManonYa09/Statistics_with_Python_G7/main/Part%204%20Regression%20Analysis/Logistics%20Regression/WA_Fn-UseC_-Telco-Customer-Churn.csv')


<a id="4"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#222222;
            overflow:hidden;
            font-weight:700">4 <span style='color:#CDA63A'>|</span>Data preprocessing and feature engineering</div>


In [14]:
columns_to_drop = ['customerID', 'gender', 'TotalCharges']

In [16]:
yes_no_columns = []
categorical_columns= []
numerical_columns = []
target = 'Churn'
for column in df.columns:
    if  column == target:
        continue
    elif column == 'customerID':
        continue
    elif column == 'TotalCharges':
        continue
    elif column == 'gender':
        continue
    elif df[column].nunique() == 2:
        yes_no_columns.append(column)
    elif df[column].nunique() <=4:
        categorical_columns.append(column)
    else:
        numerical_columns.append(column)

In [18]:
# FunctionTransformer()

In [20]:
def drop_columns(X):
    return X.drop(columns=columns_to_drop)

In [24]:
df_yes = df[df['Churn']=='Yes']
df_no = df[df['Churn']=='No']
df_no = df_no.sample(n = 1000)
data = pd.concat([df_yes, df_no])

In [26]:
x = data.drop(columns=target)
y = data[target]

In [ ]:
fit_transfrom : work , fit can work

In [128]:
preprocessing = ColumnTransformer([
    ('One_hot', OneHotEncoder(), categorical_columns),
    ('yes_no', OrdinalEncoder(), yes_no_columns),
    ('scaling', StandardScaler(), numerical_columns)])

In [130]:
traning_pipeline_logistic = Pipeline([
    ('drop', FunctionTransformer(drop_columns)),
    ('preproccessing', preprocessing),
    ('model1', LogisticRegression())
    ])

In [172]:
x_train, x_test, y_train , y_test = train_test_split(x, y, test_size=.1) ## cross validation

In [174]:
traning_pipeline_logistic.fit(x_train, y_train)

Pipeline(steps=[('drop',
                 FunctionTransformer(func=<function drop_columns at 0x1746b44a0>)),
                ('preproccessing',
                 ColumnTransformer(transformers=[('One_hot', OneHotEncoder(),
                                                  ['MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaymentMethod']),
                                                 ('yes_no', OrdinalEncoder(),
                                                  ['SeniorCitizen', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'PaperlessBilling']),
                                                 ('scaling', StandardScaler(),
                                                  ['tenure',
                                                   'MonthlyCharges'])])),
                ('model1', LogisticRegression())])

In [176]:
y_prediction_logistics = traning_pipeline_logistic.predict(x_test)

In [178]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 

In [180]:
traning_pipeline__SCV = Pipeline([
    ('drop', FunctionTransformer(drop_columns)),
    ('preproccessing', preprocessing),
    ('model', SVC())])

In [182]:
param_grid = {
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(estimator=traning_pipeline__SCV,
                    param_grid=param_grid,
                    scoring='accuracy')

In [192]:
# grid.fit(x_train,y_train)

In [185]:
y_prediction_svc = grid.predict(x_test)

In [186]:
confusion_matrix(y_test, y_prediction_logistics)

array([[ 55,  47],
       [ 23, 162]])

In [187]:
confusion_matrix(y_test, y_prediction_svc)

array([[ 49,  53],
       [ 18, 167]])

## Evaluation Metrics

In [ ]:
# y.value_counts()

In [ ]:
confusion_matrix(y_test, y_prediction)

In [ ]:
(167 + 418) / (167 + 418 + 36+ 97)

In [ ]:
(True Negatie + True +Positive) / (True Negatie + True +Positive  + False Negative False Positive)

<a id="5"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#222222;
            overflow:hidden;
            font-weight:700">4 <span style='color:#CDA63A'>|</span>Data Splitting and Model Building</div>

<a id="5.1"></a>
# <b>4.1 <span style='color:#00008B'>Data Splitting Or Cross Validation</span></b>


**splitting** our dataset into training and testing sets to avoid bias in your model.<br>
**Overfitting**

In [ ]:
X = df2.drop(columns='Churn')
y = df2['Churn']

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

<a id="5.2"></a>
# <b>5.2 <span style='color:#00008B'>Model Buiding and Model Evaluation</span></b>

**LogisticRegression**
<img src="https://raw.githubusercontent.com/ManonYa09/Statistics_with_Python_G7/main/Part%204%20Regression%20Analysis/Photos/Logistics.png" alt="Logistics" width="50%" style="display: block; margin: 0 auto;">
<img src="https://raw.githubusercontent.com/ManonYa09/Statistics_with_Python_G7/main/Part%204%20Regression%20Analysis/Photos/Sigmoid_function.png" alt="Logistics" width="50%" style="display: block; margin: 0 auto;">

<img src="https://raw.githubusercontent.com/ManonYa09/Statistics_with_Python_G7/main/Part%204%20Regression%20Analysis/Photos/confusion_matrix.webp" alt="Logistics" width="50%" style="display: block; margin: 0 auto;">

In [ ]:
def model_eval(y_test,y_pred): ## y_test:y_actual
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    error = 1-accuracy
    precision=tp/(tp+fp) ## tp : 1: correct , fp: 1 wrong
    recall=tp/(tp+fn) ## tp : 1: correct, fn: 0 : wrong, 1 
    specificity=tn/(fp+tn)
    result={"Accuracy":accuracy,"Error":error, "Precision":precision,"Recall":recall,'Specificity':specificity}
    return result

In [ ]:
model_eval(y_test2, y_pred2)

<a id="5.2.1"></a>
## <b>5.2.1 <span style='color:#00008B'>Feature selection</span></b>


<a id="5.2.1.1"></a>
### <b>5.2.1.1 <span style='color:#00008B'>Recursive Feature Elimination (RFE)</span></b>

[More about RFE](https://www.analyticsvidhya.com/blog/2023/05/recursive-feature-elimination/#:~:text=Recursive%20Feature%20Elimination%20is%20a,number%20of%20features%20is%20obtained.)
```python
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
model = LogisticRegression()

# Initialize RFE with desired number of features
n_features = 10
rfe = RFE(estimator=model, n_features_to_select=n_features)

# Fit RFE on the training data
X_train_selected = rfe.fit_transform(X_train, y_train)
X_test_selected = rfe.transform(X_test)

# Get selected feature indices
selected_feature_indices = rfe.support_
selected_features = X.columns[selected_feature_indices]

```


## Model building using RFE
---
**Recursive Feature Elimination** algorithm works in the following steps:
1. Rank the importance of all features using the chosen RFE machine learning algorithm.
2. Eliminate the least important feature.
3. Build a model using the remaining features.

In [ ]:
n = int(input('Input the number of features you want to predict: '))

rfe = RFE(Logistic2, n_features_to_select= n)
rfe.fit(x_train, y_train)

In [ ]:
len(x_train2.columns)

In [ ]:
rfe.support_

In [ ]:

selected_features = x_train[x_train.columns[rfe.support_]]

In [ ]:
selected_features

In [ ]:
# selected_features

In [ ]:
Logistic3 = LogisticRegression()

In [ ]:
# 

In [ ]:
# x_train = x_train[selected_features]

In [ ]:
Logistic3.fit(selected_features, y_train)

In [ ]:
selected_features.columns

In [ ]:
x_test2[selected_features.columns]

In [ ]:
y_pred3 = Logistic3.predict(x_test2[selected_features.columns])

In [ ]:
len(y_pred3)

In [ ]:
model_eval(y_test2, y_pred3)

In [ ]:
x_test2[selected_features.columns]

In [ ]:
Logistic3.coef_

In [ ]:
# selected_features.isnull().sum()

In [ ]:
# y_train.isnull().sum()

In [ ]:
# selected_features = selected_features.astype(int)

In [ ]:
# y_train

In [ ]:
# Logistic1.coef_

In [ ]:
selected_features

In [ ]:
coefficients1 = Logistic3.coef_
coefficients1

In [ ]:
coefficients1 = coefficients1[0].tolist()

In [ ]:
feature_names=selected_features.columns.tolist()

In [ ]:
final_featureimportance = pd.DataFrame({'Coefficient':coefficients1,
                          'feature_names1':feature_names})
final_featureimportance

In [ ]:
data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
sns.countplot(x = data['Contract'], hue = data['Churn'])

In [ ]:
sns.countplot(x = data['InternetService'], hue = data['Churn'])

In [ ]:
final_featureimportance.sort_values(by = 'Coefficient', ascending=False)

In [ ]:
# Logistic1.intercept_

In [ ]:
from explainerdashboard import ClassifierExplainer , ExplainerDashboard



In [ ]:
explainer = ClassifierExplainer(Logistic2, selected_features, y_train)
ExplainerDashboard(explainer, port=433).run()